In [ ]:
import pandas as pd
import scipy.optimize as opt
import numpy as np

In [ ]:
# read in demand data
demand = pd.read_csv("/content/strawberries_demand.csv")

# read in supply predictions data
actual_supply = pd.read_csv("/content/strawberries_var_predict.csv")

# read in prices data
wholesale_prices = pd.read_csv("/content/wholesale_prices_short.csv")
retail_prices = pd.read_csv("/content/est_retail_prices.csv")

In [ ]:
demand = demand[demand["year"] >= 1998]
demand = demand[demand["year"] <= 2015]
demand.reset_index(inplace=True, drop=True)

demand.head()

,year,Chicago,Detroit,Minneapolis.StPaul,Boston,New.York,Philadelphia,Atlanta,Baltimore,Dallas.FortWorth,Houston,Miami,DC,Los.Angeles,San.Francisco,San.Diego,Seattle,Average
0,1998,5.397224e+07,2.482775e+07,1.843363e+07,2.302365e+07,1.261287e+08,2.559082e+07,1.729105e+07,1.282960e+07,2.840600e+07,2.224338e+07,2.339995e+07,3.144440e+07,6.629119e+07,3.722528e+07,1.697203e+07,2.001542e+07,3.224089e+07
1,1999,5.142153e+07,2.511441e+07,1.835435e+07,2.662588e+07,1.218098e+08,2.973149e+07,1.910471e+07,1.468404e+07,3.363509e+07,2.480953e+07,2.713952e+07,3.183396e+07,7.092451e+07,3.739031e+07,1.702442e+07,2.228797e+07,3.364068e+07
2,2000,4.255365e+07,2.366848e+07,1.771581e+07,2.368192e+07,1.252263e+08,3.190262e+07,1.824839e+07,1.298390e+07,3.484283e+07,2.320745e+07,2.897881e+07,3.031344e+07,6.822540e+07,3.377680e+07,1.376548e+07,2.405023e+07,3.253773e+07
3,2001,3.155113e+07,1.463533e+07,1.172268e+07,1.601041e+07,8.203651e+07,2.098714e+07,1.139418e+07,8.014168e+06,2.295407e+07,1.451829e+07,1.927337e+07,1.800031e+07,4.196980e+07,2.033335e+07,7.945413e+06,1.556644e+07,2.099486e+07
4,2002,2.889577e+07,1.289088e+07,1.117663e+07,1.575836e+07,7.214225e+07,1.590736e+07,1.217339e+07,8.269800e+06,1.925882e+07,1.343695e+07,1.752912e+07,1.758757e+07,4.061988e+07,1.736761e+07,8.103688e+06,1.336223e+07,1.908708e+07


In [ ]:
actual_supply = actual_supply[actual_supply["year"] >= 1999]
actual_supply.reset_index(inplace=True, drop=True)

actual_supply.head()

,year,west_lbs,p_sw_lbs,mw_se_lbs,a_ne_lbs,west_half_lbs,east_half_lbs
0,1999,2.462539e+08,0,8.999310e+07,2.290734e+07,2.462539e+08,1.129004e+08
1,2000,3.432733e+08,0,1.158735e+08,3.136660e+07,3.432733e+08,1.472401e+08
2,2001,3.630885e+08,0,1.177554e+08,3.193696e+07,3.630885e+08,1.496924e+08
3,2002,3.165495e+08,0,1.056114e+08,2.885486e+07,3.165495e+08,1.344663e+08
4,2003,3.259869e+08,0,1.107961e+08,2.713416e+07,3.259869e+08,1.379303e+08


In [ ]:
wholesale_prices = wholesale_prices[wholesale_prices["year"] >= 1999]
wholesale_prices = wholesale_prices[wholesale_prices["year"] <= 2015]
wholesale_prices.reset_index(inplace=True, drop=True)

wholesale_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,21.3,35.0,33.00,28.6,74.4,102,17.250,11.101633,67.161655,...,16.8,24.1,32.1,15.098916,46.5,11.3,9.7,16.9,27.428610,6.9
1,2000,17.8,36.0,32.35,27.6,64.9,115,7.000,11.035726,79.001992,...,13.1,31.2,31.8,19.228349,42.6,12.3,11.2,18.5,28.129329,5.3
2,2001,22.9,38.0,34.50,30.7,75.8,109,9.625,11.863701,72.454395,...,17.1,26.5,32.4,19.287165,45.0,13.4,10.7,19.5,27.847922,10.8
3,2002,25.8,43.0,34.30,30.6,71.3,117,11.250,12.184179,87.104935,...,19.1,31.4,34.2,23.559195,47.6,12.9,12.1,19.2,27.546230,9.6
4,2003,26.1,41.0,34.45,29.1,74.9,120,8.125,12.388878,79.634487,...,19.0,32.7,36.9,21.827844,49.2,13.3,13.7,19.2,28.384431,7.3


In [ ]:
retail_prices = retail_prices[retail_prices["year"] >= 1999]
retail_prices = retail_prices[retail_prices["year"] <= 2015]
retail_prices.reset_index(inplace=True, drop=True)

retail_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,86.0,52.5,49.500,42.90,208,153.0,25.8750,16.652449,100.742482,...,25.20,36.15,48.15,22.648374,69.75,16.95,14.55,25.35,41.142914,10.35
1,2000,88.0,54.0,48.525,41.40,188,172.5,10.5000,16.553589,118.502987,...,19.65,46.80,47.70,28.842524,63.90,18.45,16.80,27.75,42.193994,7.95
2,2001,83.0,57.0,51.750,46.05,229,163.5,14.4375,17.795551,108.681593,...,25.65,39.75,48.60,28.930747,67.50,20.10,16.05,29.25,41.771883,16.20
3,2002,91.0,64.5,51.450,45.90,228,175.5,16.8750,18.276268,130.657402,...,28.65,47.10,51.30,35.338792,71.40,19.35,18.15,28.80,41.319345,14.40
4,2003,94.0,61.5,51.675,43.65,237,180.0,12.1875,18.583316,119.451730,...,28.50,49.05,55.35,32.741766,73.80,19.95,20.55,28.80,42.576646,10.95


In [ ]:
# parameters
N = 16 # number of retailers
K = demand.shape[0]-1 # number of iterations (0th one is the initial condition)

# dataframes to keep track of orders
# columns represent retailers, rows represent iterations
orders_west = pd.DataFrame(columns=range(0,N), index=range(0,K+1))
orders_mw_se = pd.DataFrame(columns=range(0,N), index=range(0,K+1))
orders_a_ne = pd.DataFrame(columns=range(0,N), index=range(0,K+1))

# initial orders
for i in range(0,N):
  orders_west.iloc[0,i] = demand["Average"].iloc[0] / 3
  orders_mw_se.iloc[0,i] = demand["Average"].iloc[0] / 3
  orders_a_ne.iloc[0,i] = demand["Average"].iloc[0] / 3

In [ ]:
def ret_cost_sim(q, k, curr_i, orders_west, orders_mw_se, orders_a_ne, demand, supply, wholesale_prices, retail_prices):
  Q_i = demand.iloc[k+1,curr_i+1]

  total_request_west = q[0]
  total_request_west += orders_west.iloc[k,0:curr_i].sum() + orders_west.iloc[k,curr_i+1:].sum()

  total_request_mw_se = q[1]
  total_request_mw_se += orders_mw_se.iloc[k,0:curr_i].sum() + orders_mw_se.iloc[k,curr_i+1:].sum()

  total_request_a_ne = q[2]
  total_request_a_ne += orders_a_ne.iloc[k,0:curr_i].sum() + orders_a_ne.iloc[k,curr_i+1:].sum()

  if (total_request_west <= supply["west_lbs"][k]):
    curr_R_west = q[0]
  else:
    curr_R_west = supply["west_lbs"][k] * (q[0] / total_request_west)

  if (total_request_mw_se <= supply["mw_se_lbs"][k]):
    curr_R_mw_se = q[1]
  else:
    curr_R_mw_se = supply["mw_se_lbs"][k] * (q[1] / total_request_mw_se)

  if (total_request_a_ne <= supply["a_ne_lbs"][k]):
    curr_R_a_ne = q[2]
  else:
    curr_R_a_ne = supply["a_ne_lbs"][k] * (q[2] / total_request_a_ne)

  R = curr_R_west + curr_R_mw_se + curr_R_a_ne

  holding = max(R - Q_i, 0)
  stockout = max(Q_i - R, 0)

  holding = wholesale_prices["Strawberries"][k] * holding
  stockout = retail_prices["Strawberries"][k] * stockout

  cost = holding + stockout

  return cost

In [ ]:
for k in range(0,K):
  print(k)

  for curr_i in range(0,N):
    rranges = (slice(10**5, 10**6, 10**5), slice(10**5, 10**6, 10**5), slice(10**5, 10**6, 10**5))
    params_curr = (k, curr_i, orders_west, orders_mw_se, orders_a_ne, demand, actual_supply, wholesale_prices, retail_prices)
    opt_sol = opt.brute(ret_cost_sim, rranges, args=params_curr, full_output=True)

    if opt_sol[0][0] > 0:
      orders_west.iloc[k+1,curr_i] = opt_sol[0][0]
    else:
      orders_west.iloc[k+1,curr_i] = 0

    if opt_sol[0][1] > 0:
      orders_mw_se.iloc[k+1,curr_i] = opt_sol[0][1]
    else:
      orders_mw_se.iloc[k+1,curr_i] = 0

    if opt_sol[0][2] > 0:
      orders_a_ne.iloc[k+1,curr_i] = opt_sol[0][2]
    else:
      orders_a_ne.iloc[k+1,curr_i] = 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [ ]:
demand["Average"]

0     3.224089e+07
1     3.364068e+07
2     3.253773e+07
3     2.099486e+07
4     1.908708e+07
5     3.128104e+07
6     2.565147e+07
7     2.511754e+07
8     2.652540e+07
9     3.205515e+07
10    3.103831e+07
11    4.364316e+07
12    3.976588e+07
13    3.364650e+07
14    3.695432e+07
15    4.506435e+07
16    4.671875e+07
17    2.945528e+07
Name: Average, dtype: float64

In [ ]:
orders_total = orders_west + orders_mw_se + orders_a_ne
(orders_total.iloc[:,1] - demand["Detroit"]) / 10**6

0     7.413136
1     5.560713
2     4.858745
3     2.313475
4     0.298827
5     1.360761
6     0.451705
7     0.807363
8     1.774263
9     1.347973
10    0.849056
11    1.379682
12    0.941891
13    0.829413
14    2.482528
15    1.534224
16    1.558516
17    0.443757
dtype: object

In [ ]:
orders_west.to_csv("/content/strawberries_var_orders_west.csv", index=False)
orders_mw_se.to_csv("/content/strawberries_var_orders_mw_se.csv", index=False)
orders_a_ne.to_csv("/content/strawberries_var_orders_a_ne.csv", index=False)